<a href="https://colab.research.google.com/github/mjgpinheiro/Econophysics/blob/main/Ergontropic_stocktrading3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

# Define system variables
ticker = 'AAPL'
start_date = '2010-01-01'
end_date = '2021-09-01'
window_size = 30
stop_loss_pct = 0.05
investment_size = 10000

# Load data from Yahoo Finance
data = yf.download(ticker, start=start_date, end=end_date)

# Calculate energy and entropy
data['Returns'] = data['Adj Close'].pct_change()
data['Energy'] = data['Returns'].rolling(window_size).apply(lambda x: np.sum(x**2))
data['Entropy'] = data['Returns'].rolling(window_size).apply(lambda x: -np.sum(x*np.log(np.abs(x))))
data.dropna(inplace=True)

# Load news sentiment data
news_sentiment = pd.read_csv('news_sentiment.csv', index_col=0)

# Merge data and news sentiment
merged_data = pd.merge(data, news_sentiment, left_index=True, right_index=True)

# Prepare features and target
features = merged_data[['Energy', 'Entropy', 'Sentiment']]
target = merged_data['Returns'].shift(-1)
features.dropna(inplace=True)
target.dropna(inplace=True)

# Scale features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Train neural network
nn = MLPRegressor(hidden_layer_sizes=(10, 10), activation='relu', solver='adam', max_iter=5000, random_state=0)
nn.fit(scaled_features, target)

# Get last day's features
last_day_features = scaled_features[-1].reshape(1, -1)

# Predict next day's return
predicted_return = nn.predict(last_day_features)

# Calculate stop-loss
stop_loss = data['Adj Close'][-1] * (1 - stop_loss_pct)

# Make trading decision based on predicted return and stop-loss
if predicted_return > 0 and data['Adj Close'][-1] < stop_loss:
    trade = 'Buy'
    shares = investment_size / data['Adj Close'][-1]
elif predicted_return < 0:
    trade = 'Sell'
    shares = investment_size / data['Adj Close'][-1]
else:
    trade = 'Hold'
    shares = 0

# Print trading decision
print(f'Trade: {trade}, Shares: {shares:.2f}, Investment Size: {investment_size:.2f}')


[*********************100%***********************]  1 of 1 completed


FileNotFoundError: ignored